In [1]:
import pandas_handler ,database_handler,lookups

In [2]:
database_handler.return_data_as_df(lookups.prediction.DISEASE.value,lookups.InputTypes.CSV)

,code,disease_name,Description
0,495,Ischemic stroke,"Interruption of blood supply to the brain, oft..."
1,500,Atrial fibrillation and flutter,Irregular and rapid heart rhythm in the upper ...
2,493,Ischemic heart disease,Reduction in blood supply to the heart muscle ...
3,496,Intracerebral hemorrhage,Bleeding within the brain due to the rupture o...
4,498,Hypertensive heart disease,High blood pressure leading to heart damage an...
5,492,Rheumatic heart disease,Heart condition resulting from untreated strep...
6,502,Peripheral artery disease,"Reduced blood flow to limbs, typically the leg..."
7,497,Subarachnoid hemorrhage,Stroke caused by bleeding into the space aroun...
8,504,Non-rheumatic valvular heart disease,Heart valve disorders not related to rheumatic...
9,503,Endocarditis,"Infection of the heart's inner lining, often c..."
